### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

In [1]:
import pandas as pd
import os
print(os.listdir("."))

td = pd.read_csv("trade_details.csv",sep=";")
tds = pd.read_csv("trade_details_snapshot.csv",sep=";")

td

['Ejercicio Pandas 1.md', 'ejercicioPandas1.ipynb', 'trade_details.csv', 'trade_details_snapshot.csv']


,mfamily,mgroup,mtype,origin_trade_number,origin_contract_number,maturity
0,IRD,BOND,NaN,316391872,678876251,2021-09-22
1,CURR,FXD,FXD,32734782,54853428,2021-09-22
2,IRD,LN_BR,NaN,1111,2222,2022-10-06
3,IRD,IRS,NaN,2222222,2222222,2024-10-15
4,SCF,SCF,SCF,3815982,3672136,NaN
5,IRD,LN_BR,NaN,14596583,13774383,2020-12-29
6,IRD,IRS,NaN,18343978,17356077,2024-10-15
7,IRD,LN_BR,NaN,19203839,18176215,2022-10-06
8,IRD,CF,NaN,20513130,19433281,2021-07-06
9,IRD,CF,NaN,20533916,19453781,2023-07-06


### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

In [2]:
import numpy as np

def obtenerFXSwaps(td,tds):
    #Obtengo los FXSwaps pero con madurity con posible valor null
    FXSwaps = td[(td.mfamily == "CURR") & (td.mgroup == "FXD") & (td.mtype == "SWLEG")].copy()
    #Obtengo la lista de indices
    indices = FXSwaps.index.tolist()
    #Como hay valores null obtengo los valores de la copia de seguridad
    FXSwaps = tds[tds.origin_trade_number.isin(FXSwaps.origin_trade_number)]
    #Le pongo los indices anteriores (los de la lista)
    FXSwaps.index = indices

    return FXSwaps

def obtenerMaxFecha(td,tds):
    FXSwaps = obtenerFXSwaps(td,tds)
    FXSwapsMax = FXSwaps.groupby(["origin_contract_number"])["maturity"].max()

    return FXSwaps[(FXSwaps.origin_contract_number.isin(FXSwapsMax.index)) & (FXSwaps.maturity.isin(FXSwapsMax.values))]

FXSwaps = obtenerMaxFecha(td,tds)
FXSwaps

,mfamily,mgroup,mtype,origin_trade_number,origin_contract_number,maturity
22,CURR,FXD,SWLEG,19665186,18622136,2020-12-30
24,CURR,FXD,SWLEG,19772400,18724280,2021-11-05
25,CURR,FXD,SWLEG,20980932,19883451,2020-02-02
28,CURR,FXD,SWLEG,22798005,21622649,2021-05-11
30,CURR,FXD,SWLEG,20665178,19622128,2020-12-30


### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

In [40]:
def obtenerMinFecha(td,tds):
    FXSwaps = obtenerFXSwaps(td,tds)
    FXSwapsMin = FXSwaps.groupby(["origin_contract_number"])["maturity"].min()

    return FXSwaps[(FXSwaps.origin_contract_number.isin(FXSwapsMin.index)) & (FXSwaps.maturity.isin(FXSwapsMin.values))]

def obtenerFechas(td,tds):
    #Obtengo los valores que se quedan fuera de el dataframe final
    noObtener = obtenerMinFecha(td,tds)
    #Obtengo el dataframe sinlos short leg
    obtener = td[~td.origin_trade_number.isin(noObtener.origin_trade_number)]

    #Obtengo las fechas vacias
    vacias = obtener[obtener.maturity.isna()]
    #A la variable le asigno el valor que les falta
    vacias = tds[tds.origin_trade_number.isin(vacias.origin_trade_number)]

    #Elimino las filas con fecha Null
    obtener = obtener.drop(labels=obtener[obtener.maturity.isna()].index,axis=0)
    #Concateno con las filas con los valores que faltan y los ordeno por indice
    obtener = pd.concat([obtener,vacias]).sort_index()

    return obtener


result = obtenerFechas(td,tds)
result

,mfamily,mgroup,mtype,origin_trade_number,origin_contract_number,maturity
0,IRD,BOND,NaN,316391872,678876251,2021-09-22
1,CURR,FXD,FXD,32734782,54853428,2021-09-22
2,IRD,LN_BR,NaN,1111,2222,2022-10-06
3,IRD,IRS,NaN,2222222,2222222,2024-10-15
4,CURR,FXD,SWLEG,20980932,19883451,2020-02-02
5,IRD,LN_BR,NaN,14596583,13774383,2020-12-29
6,IRD,IRS,NaN,18343978,17356077,2024-10-15
7,IRD,LN_BR,NaN,19203839,18176215,2022-10-06
7,CURR,FXD,SWLEG,22798005,21622649,2021-05-11
8,IRD,CF,NaN,20513130,19433281,2021-07-06
